In [49]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.metrics import roc_auc_score
import os

In [50]:
models = os.listdir('../output')
print models

['logistic', 'lightGBM_3', 'lightGBM_0', 'randomFRST', 'randomFRST_2', 'lightGBM_2', 'lightGBM_1']


In [51]:
df = pd.read_csv('../output/{0}/fold_1/oof_pred.csv'.format(models[0]))
target = df.is_attributed.values
n = len(df)
print n
print list(target).count(1)

5000001
12471


In [52]:
df = pd.read_csv('../output/{0}/fold_1/test_pred.csv'.format(models[0]))
ids = df.click_id.tolist()
n_test = len(df)
print n_test

18790469


##### Simple average

In [53]:
output_dir = '../simple_ave'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for m in models:
    print "Checking the ave of " + m
    ans = np.zeros(n)
    testp = np.zeros(n_test)
    for i in range(1, 10):
        print i,
        df = pd.read_csv('../output/{0}/fold_{1}/oof_pred.csv'.format(m, i))
        tmp = np.array(df.pred.tolist())
        print roc_auc_score(target, tmp),
        ans += 1./9. * tmp
        df = pd.read_csv('../output/{0}/fold_{1}/test_pred.csv'.format(m, i))
        testp += 1./9. * np.array(df.pred.tolist())
    score = roc_auc_score(target, ans)
    print "\n SCORE=" + str(score)
    oof_df = pd.DataFrame({
        'is_attributed': list(target),
        'pred': list(ans)
    })
    oof_df = oof_df[['is_attributed', 'pred']]
    oof_df.to_csv(output_dir + '/' + m + '_oof_{SCORE}.csv'.format(SCORE=str(score)), index=False)
    test_df = pd.DataFrame({
        'click_id': ids,
        'is_attributed': testp
    })
    test_df['click_id'] = test_df['click_id'].astype(int)
    test_df = test_df[['click_id', 'is_attributed']]
    test_df.to_csv(output_dir + '/' + m + '_test.csv', index=False)

Checking the ave of logistic
1 0.9124174276 2 0.91249590902 3 0.911643691471 4 0.912530471636 5 0.911769611401 6 0.912087015757 7 0.911735940159 8 0.912116817579 9 0.911814780086 
 SCORE=0.912809181445
Checking the ave of lightGBM_3
1 0.975408758338 2 0.975515261339 3 0.975578902017 4 0.975436421283 5 0.975315179454 6 0.975047248939 7 0.975409852503 8 0.975364415994 9 0.974974347694 
 SCORE=0.976315623129
Checking the ave of lightGBM_0
1 0.974458964204 2 0.974579614774 3 0.974298369631 4 0.974639027306 5 0.974355254007 6 0.974252339806 7 0.974368777768 8 0.974533317012 9 0.97395381743 
 SCORE=0.975372260552
Checking the ave of randomFRST
1 0.968584547777 2 0.968613800856 3 0.967582925121 4 0.967970539446 5 0.967527060014 6 0.967907333169 7 0.968450162633 8 0.968043676641 9 0.96746591914 
 SCORE=0.971250412087
Checking the ave of randomFRST_2
1 0.96754824252 2 0.966985202479 3 0.967075828034 4 0.966459657932 5 0.967126319713 6 0.967194781781 7 0.967367484734 8 0.968114272328 9 0.9664312

### Exponential Average

In [ ]:
delta = 1E-10
def sigmoid(x):
    return 1. / (np.exp(-x)+1.)
def upper(x):
    return min(x, 1.-delta)
def lower(x):
    return max(x, delta)
def rev(x):
    return -np.log(1./x - 1.)

In [ ]:
output_dir = '../exponential_ave'
print len(target)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for m in models:
    print "Checking the ave of " + m
    ans = np.zeros(n)
    testp = np.zeros(n_test)
    for i in range(1, 10):
        print i,
        df = pd.read_csv('../output/{0}/fold_{1}/oof_pred.csv'.format(m, i))
        df.pred = df.pred.apply(upper)
        df.pred = df.pred.apply(lower)
        df.pred = df.pred.apply(rev)
        tmp = np.array(df.pred.tolist())
        print roc_auc_score(target, sigmoid(tmp)),
        ans += 1./9. * tmp
        df = pd.read_csv('../output/{0}/fold_{1}/test_pred.csv'.format(m, i))
        df.pred = df.pred.apply(upper)
        df.pred = df.pred.apply(lower)
        df.pred = df.pred.apply(rev)
        testp += 1./9. * np.array(df.pred.tolist())
    ans = sigmoid(ans)
    testp = sigmoid(testp)
    score = roc_auc_score(target, ans)
    print "\n SCORE=" + str(score)
    oof_df = pd.DataFrame({
        'is_attributed': list(target),
        'pred': list(ans)
    })
    oof_df = oof_df[['is_attributed', 'pred']]
    oof_df.to_csv(output_dir + '/' + m + '_oof_{SCORE}.csv'.format(SCORE=str(score)), index=False)
    test_df = pd.DataFrame({
        'click_id': ids,
        'is_attributed': testp
    })
    test_df['click_id'] = test_df['click_id'].astype(int)
    test_df = test_df[['click_id', 'is_attributed']]
    test_df.to_csv(output_dir + '/' + m + '_test.csv', index=False)

5000001
Checking the ave of logistic
1 0.9124174276 2 0.91249590902 3 0.911643691471 4 0.912530471636 5 0.911769611401 6 0.912087015757 7 0.911735940159 8 0.912116817579 9 0.911814780086 
 SCORE=0.912763298011
Checking the ave of lightGBM_3
1 0.975408758338 2 0.975515261339 3 0.975578902017 4 0.975436421283 5 0.975315179454 6 0.975047248939 7 0.975409852503 8 0.975364415994 9 0.974974347694 
 SCORE=0.976264602773
Checking the ave of lightGBM_0
1 0.974458964204 2 0.974579614774 3 0.974298369631 4 0.974639027306 5 0.974355254007 6 0.974252339806 7 0.974368777768 8 0.974533317012 9 0.97395381743 
 SCORE=0.975416070448
Checking the ave of randomFRST
1 0.968584547777 2 0.968613800856 3 0.967582925121 4 0.967970539446 5 0.967527060014 6 0.967907333169 7 0.968450162633 8 0.968043676641 9 0.96746591914 
 SCORE=0.970591719946
Checking the ave of randomFRST_2
1 0.96754824252 2 0.966985202479 3 0.967075828034 4 0.966459657932 5 0.967126319713 6 0.967194781781 7 0.967367484734 8 0.968114272328 9 0

##### Grid Search

In [ ]:
def gridSearch(vecs, reso=0.02):
    assert len(vecs) == 9
    coeff = np.zeros(9)
    coeff[0] = 1.
    score = roc_auc_score(target, np.dot(coeff, vecs))
    for c1 in np.arange(0, 1., reso):
        print c1,
        for c2 in np.arange(0, 1.-c1, reso):
            for c3 in np.arange(0, 1.-c1-c2, reso):
                for c4 in np.arange(0, 1.-c1-c2-c3, reso):
                    for c5 in np.arange(0, 1.-c1-c2-c3-c4, reso):
                        for c6 in np.arange(0, 1.-c1-c2-c3-c4-c5, reso):
                            for c7 in np.arange(0, 1.-c1-c2-c3-c4-c5-c6, reso):
                                for c8 in np.arange(0, 1.-c1-c2-c3-c4-c5-c6-c7, reso):
                                    c9 = 1.-c1-c2-c3-c4-c5-c6-c7-c8
                                    tmp_coeff = np.array([c1, c2, c3, c4, c5, c6, c7, c8, c9])
                                    tmp_score = roc_auc_score(target, np.dot(tmp_coeff, vecs))
                                    if tmp_score > score:
                                        coeff = tmp_coeff
                                        score = tmp_score
    return coeff, score